#### Calculating Risk parameters on the bank balance
See for a practical guide: https://www.closemountain.com/papers/risktransform1.pdf

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from src.data.zerocurve import Zerocurve
from src.data.interest import Interest
from src.models.bank_model import Bankmodel
import pandas as pd
import numpy as np


In [2]:
zerocurve = Zerocurve()
zerocurve.load_data()
interest = Interest()
interest.load_data()

2023-03-20 13:21:56.236 | INFO     | src.data.dataset:load_data:45 - loading zerocurve data from file.
c:\Users\M\miniconda3\envs\rlalm\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)
2023-03-20 13:21:56.448 | INFO     | src.data.dataset:load_data:45 - loading interest data from file.


In [3]:
pos_date = zerocurve.df.index[-1]
bankmodel = Bankmodel(pos_date)
bankmodel.generate_mortgage_contracts(n = 250, df_i = interest.df)

2023-03-20 13:22:02.628 | INFO     | src.models.bank_model:generate_mortgage_contracts:121 - Added 250 mortgages to our portfolio.
2023-03-20 13:22:07.066 | INFO     | src.models.bank_model:generate_mortgage_contracts:125 - Added 44194 cashflows to our model.


,category,fixed_period,years,start_date,principal,period,interest,contract
0,2,5>10 years,10,2021-03-31,100000,2021-03-01,1.68,0
1,3,>10 years,20,2021-08-02,100000,2021-08-01,1.75,1
2,3,>10 years,20,2014-08-18,100000,2014-08-01,4.13,2
3,3,>10 years,20,2010-04-12,100000,2010-04-01,3.15,3
4,2,5>10 years,10,2018-11-19,100000,2018-11-01,2.39,4
...,...,...,...,...,...,...,...,...
245,2,5>10 years,10,2020-12-04,100000,2020-12-01,1.75,245
246,3,>10 years,20,2004-05-13,100000,2004-05-01,3.15,246
247,3,>10 years,20,2019-06-24,100000,2019-06-01,2.76,247
248,2,5>10 years,10,2020-12-07,100000,2020-12-01,1.75,248


In [4]:
result = bankmodel.calculate_npv(zerocurve)
print(f'Net present value {result:,.0f}')

Net present value 19,420,731


##### The standardized framework. 
Supervisory Outlier Test (SOT)

Interest Rate Risk needs to be measured on time buckets
https://www.bis.org/bcbs/publ/d319.pdf
https://www.eba.europa.eu/sites/default/documents/files/document_library/Publications/Draft%20Technical%20Standards/2022/EBA-RTS-2022-10%20RTS%20on%20SOTs/1041756/Final%20draft%20RTS%20on%20SOTs.pdf

1. parallel shock up; 
2. parallel shock down;
3. steepener shock (short rates down and long rates up);
4. flattener shock (short rates up and long rates down); 
5. short rates shock up; and
6. short rates shock down

In [5]:
result = bankmodel.calculate_risk(zerocurve,200,'parallel')
print(f'Parallell shift 200bp up {result:,.0f}')

result = bankmodel.calculate_risk(zerocurve,-200,'parallel')
print(f'Parallell shift 200bp down {result:,.0f}')

result = bankmodel.calculate_risk(zerocurve,250,'short')
print(f'Short shift 250bp up {result:,.0f}')

result = bankmodel.calculate_risk(zerocurve,100,'long')
print(f'Long shift 100bp up {result:,.0f}')


Parallell shift 200bp up -1,869,718
Parallell shift 200bp down 2,281,212
Short shift 250bp up -86,197
Long shift 100bp up -944,895


Calculate BPV profile

In [6]:
# Cashflows binned per tenor
bins =  [ pos_date + pd.offsets.DateOffset(months=item) for item in zerocurve.df.loc[pos_date].tenor.to_list()]
if bankmodel.df_cashflows['value_dt'].max() > bins[-1]:
    bins.append(max(bankmodel.df_cashflows['value_dt'].max()))
cats = zerocurve.df.loc[pos_date].tenor.to_list()[1:] # list(range(1, len(bins)))

df = bankmodel.df_cashflows
df['tenor'] = pd.cut(df['value_dt'], bins, labels=cats, right=False)
df = df[df["value_dt"] > pos_date] 
df = df[['tenor','cashflow']].groupby('tenor').sum('cashflow')

df_zerocurve_date = zerocurve.df.loc[pos_date]
shock = 1 / 100
cashflow = df['cashflow'].to_numpy().reshape(-1,1)
cashflow = np.r_[ np.zeros((1,1)), cashflow ]
t = (df_zerocurve_date['tenor'] / 12).to_numpy().reshape(-1, 1)
rates = df_zerocurve_date['rate'].to_numpy()
shock_range = [-shock, shock]
new_cols = np.zeros((len(rates), len(rates)*len(shock_range)))
for s in shock_range:    
    for i in range(len(rates)):        
        new_cols[:, i + shock_range.index(s)*len(rates)] = rates
        new_cols[i, i + shock_range.index(s)*len(rates)] = rates[i]+s            
rates = rates.reshape(len(rates),1)
rates = np.concatenate((rates,new_cols), axis = 1)
discount_factor = (1 / (1 + rates / 100)) ** t
npv = discount_factor * cashflow
plus = npv[:, 1:len(rates)+1]
neg  = npv[:, len(rates)+1:]
result = np.sum(np.round(np.minimum(plus, neg) - npv[:, 0].reshape(-1, 1),0), axis = 0).reshape(-1, 1)
df_result = pd.DataFrame(result)
df_result['tenor'] = df_zerocurve_date['tenor'].to_list()
df_result.set_index('tenor', inplace = True)
df_result.columns = ['dv01']
df_result

df_curves = pd.DataFrame(rates)
df_curves['tenor'] = df_zerocurve_date['tenor'].to_list()
df_curves.set_index('tenor', inplace = True)
df_discount_factor = pd.DataFrame(discount_factor)
df_discount_factor['tenor'] = df_zerocurve_date['tenor'].to_list()
df_discount_factor.set_index('tenor', inplace = True)
df_npv = pd.DataFrame(npv)
df_npv['tenor'] = df_zerocurve_date['tenor'].to_list()
df_npv.set_index('tenor', inplace = True)
df_result = pd.DataFrame(result)
df_result['tenor'] = df_zerocurve_date['tenor'].to_list()
df_result.set_index('tenor', inplace = True)
print (df_curves.head())
df_curves.to_excel('df_curves.xlsx')
print (df_npv.head())
df_npv.to_excel('df_npv.xlsx')
print (df_discount_factor.head())
df_discount_factor.to_excel('df_discount_factor.xlsx')
print (df_result.head())
df_result.to_excel('df_result.xlsx')

             0         1         2         3         4         5         6   \
tenor                                                                         
0      3.052459  3.042459  3.052459  3.052459  3.052459  3.052459  3.052459   
3      3.052459  3.052459  3.042459  3.052459  3.052459  3.052459  3.052459   
6      3.369500  3.369500  3.369500  3.359500  3.369500  3.369500  3.369500   
9      3.390191  3.390191  3.390191  3.390191  3.380191  3.390191  3.390191   
12     3.282758  3.282758  3.282758  3.282758  3.282758  3.272758  3.282758   

             7         8         9   ...        21        22        23  \
tenor                                ...                                 
0      3.052459  3.052459  3.052459  ...  3.052459  3.052459  3.052459   
3      3.052459  3.052459  3.052459  ...  3.052459  3.052459  3.052459   
6      3.369500  3.369500  3.369500  ...  3.369500  3.369500  3.369500   
9      3.390191  3.390191  3.390191  ...  3.390191  3.390191  3.390191   
12